In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset6') 

# Actions that we try to detect
actions = np.array(['Good', 'Name', 'Bad'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'Good': 0, 'Name': 1, 'Bad': 2}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
np.array(sequences).shape

(90, 30, 1662)

In [15]:
np.array(labels).shape

(90,)

In [16]:
X = np.array(sequences)

In [17]:
X.shape

(90, 30, 1662)

In [18]:
y = to_categorical(labels).astype(int)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
y_test.shape

(5, 3)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 6s 857ms/step - loss: 2.5738 - categorical_accuracy: 0.3294
Epoch 2/2000
3/3 [==============================] - 1s 303ms/step - loss: 1.8043 - categorical_accuracy: 0.2941
Epoch 3/2000
3/3 [==============================] - 1s 376ms/step - loss: 4.2498 - categorical_accuracy: 0.3529
Epoch 4/2000
3/3 [==============================] - 1s 385ms/step - loss: 2.3935 - categorical_accuracy: 0.3882
Epoch 5/2000
3/3 [==============================] - 1s 227ms/step - loss: 1.3427 - categorical_accuracy: 0.2588
Epoch 6/2000
3/3 [==============================] - 1s 313ms/step - loss: 1.3871 - categorical_accuracy: 0.4235
Epoch 7/2000
3/3 [==============================] - 1s 312ms/step - loss: 9.1950 - categorical_accuracy: 0.3765
Epoch 8/2000
3/3 [==============================] - 1s 331ms/step - loss: 5.0564 - categorical_accuracy: 0.3294
Epoch 9/2000
3/3 [==============================] - 1s 235ms/step - loss: 3.0240 - categorical_accuracy:

3/3 [==============================] - 0s 98ms/step - loss: 0.4396 - categorical_accuracy: 0.7059
Epoch 146/2000
3/3 [==============================] - 0s 126ms/step - loss: 0.3990 - categorical_accuracy: 0.7765
Epoch 147/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.5428 - categorical_accuracy: 0.6706
Epoch 148/2000
3/3 [==============================] - 0s 141ms/step - loss: 0.5267 - categorical_accuracy: 0.7059
Epoch 149/2000
3/3 [==============================] - 0s 135ms/step - loss: 0.3495 - categorical_accuracy: 0.8235
Epoch 150/2000
3/3 [==============================] - 1s 172ms/step - loss: 0.3601 - categorical_accuracy: 0.7765
Epoch 151/2000
3/3 [==============================] - 0s 179ms/step - loss: 0.3602 - categorical_accuracy: 0.8353
Epoch 152/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.3983 - categorical_accuracy: 0.7059
Epoch 153/2000
3/3 [==============================] - 0s 95ms/step - loss: 0.5063 - categorical_accuracy:

3/3 [==============================] - 1s 244ms/step - loss: 9.0083e-04 - categorical_accuracy: 1.0000
Epoch 290/2000
3/3 [==============================] - 0s 113ms/step - loss: 8.9506e-04 - categorical_accuracy: 1.0000
Epoch 291/2000
3/3 [==============================] - 0s 94ms/step - loss: 8.6350e-04 - categorical_accuracy: 1.0000
Epoch 292/2000
3/3 [==============================] - 0s 97ms/step - loss: 8.2590e-04 - categorical_accuracy: 1.0000
Epoch 293/2000
3/3 [==============================] - 0s 103ms/step - loss: 8.0295e-04 - categorical_accuracy: 1.0000
Epoch 294/2000
3/3 [==============================] - 0s 84ms/step - loss: 7.8050e-04 - categorical_accuracy: 1.0000
Epoch 295/2000
3/3 [==============================] - 0s 134ms/step - loss: 7.5598e-04 - categorical_accuracy: 1.0000
Epoch 296/2000
3/3 [==============================] - 0s 91ms/step - loss: 7.4370e-04 - categorical_accuracy: 1.0000
Epoch 297/2000
3/3 [==============================] - 1s 164ms/step - loss:

3/3 [==============================] - 1s 226ms/step - loss: 1.1920e-04 - categorical_accuracy: 1.0000
Epoch 428/2000
3/3 [==============================] - 0s 131ms/step - loss: 1.1732e-04 - categorical_accuracy: 1.0000
Epoch 429/2000
3/3 [==============================] - 0s 101ms/step - loss: 1.1627e-04 - categorical_accuracy: 1.0000
Epoch 430/2000
3/3 [==============================] - 0s 105ms/step - loss: 1.1511e-04 - categorical_accuracy: 1.0000
Epoch 431/2000
3/3 [==============================] - 1s 171ms/step - loss: 1.1435e-04 - categorical_accuracy: 1.0000
Epoch 432/2000
3/3 [==============================] - 0s 65ms/step - loss: 1.1306e-04 - categorical_accuracy: 1.0000
Epoch 433/2000
3/3 [==============================] - 1s 232ms/step - loss: 1.1177e-04 - categorical_accuracy: 1.0000
Epoch 434/2000
3/3 [==============================] - 1s 259ms/step - loss: 1.1074e-04 - categorical_accuracy: 1.0000
Epoch 435/2000
3/3 [==============================] - 0s 110ms/step - lo

3/3 [==============================] - 1s 191ms/step - loss: 4.8136e-05 - categorical_accuracy: 1.0000
Epoch 565/2000
3/3 [==============================] - 0s 131ms/step - loss: 4.7959e-05 - categorical_accuracy: 1.0000
Epoch 566/2000
3/3 [==============================] - 1s 300ms/step - loss: 4.7736e-05 - categorical_accuracy: 1.0000
Epoch 567/2000
3/3 [==============================] - 0s 116ms/step - loss: 4.7495e-05 - categorical_accuracy: 1.0000
Epoch 568/2000
3/3 [==============================] - 0s 105ms/step - loss: 4.7293e-05 - categorical_accuracy: 1.0000
Epoch 569/2000
3/3 [==============================] - 1s 181ms/step - loss: 4.6943e-05 - categorical_accuracy: 1.0000
Epoch 570/2000
3/3 [==============================] - 0s 78ms/step - loss: 4.6800e-05 - categorical_accuracy: 1.0000
Epoch 571/2000
3/3 [==============================] - 0s 89ms/step - loss: 4.6573e-05 - categorical_accuracy: 1.0000
Epoch 572/2000
3/3 [==============================] - 0s 105ms/step - los

...

3/3 [==============================] - 0s 99ms/step - loss: 3.0153e-06 - categorical_accuracy: 1.0000
Epoch 1388/2000
3/3 [==============================] - 0s 68ms/step - loss: 3.0083e-06 - categorical_accuracy: 1.0000
Epoch 1389/2000
3/3 [==============================] - 0s 60ms/step - loss: 3.0027e-06 - categorical_accuracy: 1.0000
Epoch 1390/2000
3/3 [==============================] - 1s 230ms/step - loss: 2.9942e-06 - categorical_accuracy: 1.0000
Epoch 1391/2000
3/3 [==============================] - 0s 91ms/step - loss: 2.9872e-06 - categorical_accuracy: 1.0000
Epoch 1392/2000
3/3 [==============================] - 0s 132ms/step - loss: 2.9816e-06 - categorical_accuracy: 1.0000
Epoch 1393/2000
3/3 [==============================] - 0s 115ms/step - loss: 2.9774e-06 - categorical_accuracy: 1.0000
Epoch 1394/2000
3/3 [==============================] - 1s 158ms/step - loss: 2.9690e-06 - categorical_accuracy: 1.0000
Epoch 1395/2000
3/3 [==============================] - 0s 84ms/step 

3/3 [==============================] - 0s 83ms/step - loss: 2.1135e-06 - categorical_accuracy: 1.0000
Epoch 1525/2000
3/3 [==============================] - 0s 127ms/step - loss: 2.0981e-06 - categorical_accuracy: 1.0000
Epoch 1526/2000
3/3 [==============================] - 0s 175ms/step - loss: 2.0953e-06 - categorical_accuracy: 1.0000
Epoch 1527/2000
3/3 [==============================] - 0s 78ms/step - loss: 2.0869e-06 - categorical_accuracy: 1.0000
Epoch 1528/2000
3/3 [==============================] - 0s 59ms/step - loss: 2.0826e-06 - categorical_accuracy: 1.0000
Epoch 1529/2000
3/3 [==============================] - 0s 67ms/step - loss: 2.0812e-06 - categorical_accuracy: 1.0000
Epoch 1530/2000
3/3 [==============================] - 0s 112ms/step - loss: 2.0770e-06 - categorical_accuracy: 1.0000
Epoch 1531/2000
3/3 [==============================] - 0s 104ms/step - loss: 2.0644e-06 - categorical_accuracy: 1.0000
Epoch 1532/2000
3/3 [==============================] - 0s 101ms/step

3/3 [==============================] - 0s 188ms/step - loss: 1.4698e-06 - categorical_accuracy: 1.0000
Epoch 1662/2000
3/3 [==============================] - 0s 149ms/step - loss: 1.4712e-06 - categorical_accuracy: 1.0000
Epoch 1663/2000
3/3 [==============================] - 1s 199ms/step - loss: 1.4656e-06 - categorical_accuracy: 1.0000
Epoch 1664/2000
3/3 [==============================] - 0s 56ms/step - loss: 1.4614e-06 - categorical_accuracy: 1.0000
Epoch 1665/2000
3/3 [==============================] - 0s 91ms/step - loss: 1.4586e-06 - categorical_accuracy: 1.0000
Epoch 1666/2000
3/3 [==============================] - 0s 158ms/step - loss: 1.4529e-06 - categorical_accuracy: 1.0000
Epoch 1667/2000
3/3 [==============================] - 0s 83ms/step - loss: 1.4529e-06 - categorical_accuracy: 1.0000
Epoch 1668/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.4473e-06 - categorical_accuracy: 1.0000
Epoch 1669/2000
3/3 [==============================] - 0s 99ms/step

3/3 [==============================] - 0s 118ms/step - loss: 1.0308e-06 - categorical_accuracy: 1.0000
Epoch 1799/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.0322e-06 - categorical_accuracy: 1.0000
Epoch 1800/2000
3/3 [==============================] - 1s 211ms/step - loss: 1.0294e-06 - categorical_accuracy: 1.0000
Epoch 1801/2000
3/3 [==============================] - 0s 116ms/step - loss: 1.0224e-06 - categorical_accuracy: 1.0000
Epoch 1802/2000
3/3 [==============================] - 0s 85ms/step - loss: 1.0196e-06 - categorical_accuracy: 1.0000
Epoch 1803/2000
3/3 [==============================] - 0s 116ms/step - loss: 1.0154e-06 - categorical_accuracy: 1.0000
Epoch 1804/2000
3/3 [==============================] - 0s 141ms/step - loss: 1.0098e-06 - categorical_accuracy: 1.0000
Epoch 1805/2000
3/3 [==============================] - 1s 481ms/step - loss: 1.0126e-06 - categorical_accuracy: 1.0000
Epoch 1806/2000
3/3 [==============================] - 0s 90ms/st

3/3 [==============================] - 0s 92ms/step - loss: 7.1806e-07 - categorical_accuracy: 1.0000
Epoch 1937/2000
3/3 [==============================] - 0s 140ms/step - loss: 7.1385e-07 - categorical_accuracy: 1.0000
Epoch 1938/2000
3/3 [==============================] - 0s 92ms/step - loss: 7.0824e-07 - categorical_accuracy: 1.0000
Epoch 1939/2000
3/3 [==============================] - 0s 91ms/step - loss: 7.0684e-07 - categorical_accuracy: 1.0000
Epoch 1940/2000
3/3 [==============================] - 0s 87ms/step - loss: 7.0403e-07 - categorical_accuracy: 1.0000
Epoch 1941/2000
3/3 [==============================] - 1s 218ms/step - loss: 6.9843e-07 - categorical_accuracy: 1.0000
Epoch 1942/2000
3/3 [==============================] - 0s 77ms/step - loss: 6.9282e-07 - categorical_accuracy: 1.0000
Epoch 1943/2000
3/3 [==============================] - 0s 84ms/step - loss: 6.9422e-07 - categorical_accuracy: 1.0000
Epoch 1944/2000
3/3 [==============================] - 0s 117ms/step -

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [27]:
res = model.predict(X_test)

In [28]:
actions[np.argmax(res[4])]

'Name'

In [29]:
actions[np.argmax(y_test[4])]

'Name'

In [30]:
model.save('mod6.h5')

In [31]:
model.load_weights('mod6.h5')

In [32]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:
yhat = model.predict(X_test)

In [34]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [0, 3]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [36]:
accuracy_score(ytrue, yhat)

1.0

In [37]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [38]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Name
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Name
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Name
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class '

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Good
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Bad
<class 'medi